# Tackle Noise with Error Correction

In [ ]:
# python -m venv qiskit_env
 
#  qiskit_env\Scripts\activate  

# pip install qiskit 
# pip install jupyter
# pip install qiskit-aer  
# pip install matplotlib 
# pip install qiskit-ibm-runtime


# Import necessary libraries from Qiskit

# QuantumCircuit: Used to build quantum circuits by applying quantum gates.
# assemble: Combines quantum circuits into a format that the simulator or quantum computer can process.
# Aer: Qiskit's simulator provider, which includes 'qasm_simulator' for running quantum circuits.
# transpile: Optimizes the quantum circuit for a specific backend.
# plot_histogram: Visualizes the measurement results in a histogram.
# CompleteMeasFitter, complete_meas_cal: Tools from Qiskit Ignis used for error mitigation.
from qiskit import QuantumCircuit, assemble, Aer, transpile
from qiskit.visualization import plot_histogram
from qiskit.ignis.mitigation.measurement import CompleteMeasFitter, complete_meas_cal

# Step 1: Define the Quantum Circuit
# ----------------------------------
# QuantumCircuit(3, 3) initializes a circuit with 3 qubits and 3 classical bits.
qc = QuantumCircuit(3, 3)

# Apply gates to create an entangled state
qc.h(0)        # Apply Hadamard gate to qubit 0 to create superposition
qc.cx(0, 1)    # Apply CNOT gate with qubit 0 as control and qubit 1 as target
qc.cx(0, 2)    # Apply CNOT gate with qubit 0 as control and qubit 2 as target

# Measure qubits into classical bits to record results
# In quantum computing, the qubits’ states are usually in a quantum superposition. 
# But to get a result that we can interpret (like a classical bitstring such as 000, 011, etc.), 
# we need to "collapse" the qubits’ states to a specific, measurable outcome.

# This "collapse" is achieved by measuring the qubits. The measure operation forces each qubit into either 
# ∣0⟩ or ∣1⟩ based on its quantum state probabilities, producing a classical result for each qubit.

# qc.measure([0, 1, 2], [0, 1, 2]) specifies that:
# The qubits 0, 1, and 2 should each be measured.
# The classical bits 0, 1, and 2 should store the results of measuring the respective qubits.

# qc.measure(qubit_indices, classical_bit_indices)
qc.measure([0, 1, 2], [0, 1, 2]) 
# qc.measure(range(3), range(3)) instead of qc.measure([0, 1, 2], [0, 1, 2]), and it will work exactly the same way


# In quantum computing, a circuit’s quantum operations (gates) are performed on qubits, 
# but the result of the computation is probabilistic and cannot be directly accessed. 
# The measure function is essential to retrieve these results, 
# making quantum information readable in a classical format (like 0s and 1s).



# Step 2: Transpile the Circuit for a Specific Backend
# ----------------------------------------------------
# Transpilation optimizes the circuit based on the backend (here, 'qasm_simulator')
# Aer: A module in Qiskit that provides a set of simulators for running quantum circuits on classical hardware.
# qasm_simulator: One of Aer’s simulators, designed to mimic how quantum circuits would run on an actual quantum computer.
# It uses a classical algorithm to simulate the behavior of a quantum computer and 
# outputs measurement results as bitstrings (e.g., 000, 101).
# It’s named qasm because it operates on QASM (Quantum Assembly Language),
# a low-level language for describing quantum circuits.
backend = Aer.get_backend('qasm_simulator')    # Select simulator backend for running the circuit

# Transpilation: This is an optimization process for a quantum circuit to make it compatible with a specific backend (simulator or real quantum device).
# Quantum hardware has limitations, such as a limited number of qubit connections and types of supported gates.
# Transpilation rearranges and simplifies the circuit’s gates to meet the constraints of the backend.
transpiled_qc = transpile(qc, backend)         # Transpile for optimization on the selected backend

# transpile(qc, backend) optimizes the circuit qc so it can run efficiently on qasm_simulator.


# Step 3: Simulate the Circuit Without Noise Mitigation
# -----------------------------------------------------
# Assemble the transpiled circuit to prepare for execution, and specify shots (repetitions) for statistical accuracy
qobj = assemble(transpiled_qc, shots=1000)     
# Assemble the transpiled circuit for execution, with 1000 shots
job = backend.run(qobj)                        
# Run the job on the simulator
result = job.result()                          
# Get the result from the executed job
counts = result.get_counts()                   
# Get the count of each outcome (bitstring) from the result


# assemble: Converts the transpiled quantum circuit (transpiled_qc) into a format (quantum object or qobj) that the simulator or quantum computer can run.
# shots=1000: Sets the number of times the circuit will be run (repeated) in the simulation.
# Repeating the circuit multiple times allows for a better estimate of the probability distribution of the possible outcomes.
# Here, shots=1000 means we’ll run the circuit 1,000 times to get statistically meaningful results.

#  job = backend.run(qobj)
# backend.run: Sends the assembled quantum object (qobj) to the simulator (our backend) to be executed.
# job: Stores the information about this "job," allowing you to track or retrieve the results after execution.

# result = job.result()
# job.result(): Retrieves the execution results from the simulator once the job is complete.
# result: Stores the data returned from running the circuit, including information about each measurement outcome.

# counts = result.get_counts()
# result.get_counts(): Extracts the measurement results as a dictionary, where each unique outcome (bitstring) is a key, and its frequency (number of occurrences) across the 1,000 shots is the value.
# Example: {'000': 500, '111': 500} would mean that the outcomes 000 and 111 each appeared 500 times out of 1,000 shots.


# Step 4: Set Up and Run Measurement Error Mitigation
# ---------------------------------------------------
# Measurement error mitigation reduces the effect of measurement errors on the result

# 1. Generate calibration circuits for measurement error mitigation
# - complete_meas_cal returns calibration circuits and state labels.
# - qubit_list specifies which qubits to calibrate for measurement error.
cal_circuits, state_labels = complete_meas_cal(qubit_list=[0, 1, 2])

# 2. Run calibration circuits to get calibration results
# - These results help to calibrate and later filter out measurement errors.
cal_job = backend.run(assemble(cal_circuits, backend=backend))
cal_results = cal_job.result()

# 3. Use CompleteMeasFitter to create an error mitigation filter from calibration results
# - CompleteMeasFitter is initialized with calibration results and state labels.
# - The filter can later be used to "correct" measurement errors in the results.
meas_fitter = CompleteMeasFitter(cal_results, state_labels)

# Step 5: Apply Error Mitigation Filter
# -------------------------------------
# Use the measurement fitter to mitigate errors in the original counts
mitigated_counts = meas_fitter.filter.apply(counts)

# Display Results
# ---------------
print(f"Original Counts : {counts}")          # Print the raw, uncorrected counts
print(f"Mitigated Counts : {mitigated_counts}") # Print the counts after error mitigation

# Visualize Results
# -----------------
# plot_histogram displays measurement results for visual comparison.
# - Pass both original and mitigated counts with a legend to distinguish them.
plot_histogram([counts, mitigated_counts], legend=['Original', 'Mitigated'])



# 

# Why Errors occur

# Decoherence: Qubits lose their quantum state over time due to interactions with their environment.
# Gate Errors: Quantum gates (operations on qubits) aren't perfectly accurate and introduce small mistakes.
# Measurement Errors: When we measure qubits, noise in the hardware can cause incorrect readings.

# To prevent or reduce these types of quantum errors:

# Decoherence: Use high-quality qubits that are better isolated from their environment and improve cooling techniques to keep qubits stable longer.

# Gate Errors: Optimize circuit designs to use fewer gates and employ calibration techniques to improve gate accuracy.

# Measurement Errors: Apply measurement error correction by running calibration circuits, which help adjust for known inaccuracies in measurements.

# Quantum Error Correction Codes: Use extra qubits to create redundancy, so errors can be detected and corrected without disrupting the computation.

# Improved Hardware and Materials: Invest in higher precision materials and more stable quantum processors to minimize all types of errors.

ImportError: Qiskit is installed in an invalid environment that has both Qiskit >=1.0 and an earlier version. You should create a new virtual environment, and ensure that you do not mix dependencies between Qiskit <1.0 and >=1.0. Any packages that depend on 'qiskit-terra' are not compatible with Qiskit 1.0 and will need to be updated. Qiskit unfortunately cannot enforce this requirement during environment resolution. See https://qisk.it/packaging-1-0 for more detail.